In [9]:
# Cell 1: Setup Working Directory
import os
import sys

# Change to project root if we're in notebooks folder
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
    print(f"✓ Changed to project root: {os.getcwd()}")
else:
    print(f"✓ Already in project root: {os.getcwd()}")

# Add src to path
sys.path.insert(0, 'src')
print(f"✓ Python path updated")

✓ Already in project root: d:\MINESTUDY\Research\idiom3.0\idiom3.0
✓ Python path updated


In [10]:
# Cell 2: Imports
from inference import (
    load_trained_model,
    translate,
    batch_translate,
    translate_with_idiom
)
from trainer import load_config
import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

print("✓ Imports successful")

✓ Imports successful


In [11]:
# Cell 3: Load configuration
config = load_config('config/training_config.yaml')

print("✓ Configuration loaded")
print(f"Source language: {config['model']['source_lang']}")
print(f"Target language: {config['model']['target_lang']}")

✓ Configuration loaded
Source language: eng_Latn
Target language: sin_Sinh


In [12]:
# Cell 4: Find and load the latest checkpoint
checkpoint_dir = 'models/checkpoints'

# Check if checkpoint directory exists
if not os.path.exists(checkpoint_dir):
    raise FileNotFoundError(f"Checkpoint directory not found: {checkpoint_dir}")

checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]

if not checkpoints:
    raise FileNotFoundError("No checkpoints found! Training may not have completed.")

# Get the latest checkpoint (highest step number)
latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[1]))[-1]
model_path = os.path.join(checkpoint_dir, latest_checkpoint)

print(f"Found checkpoints: {checkpoints}")
print(f"Loading latest checkpoint: {model_path}")

# Load the model
model, tokenizer = load_trained_model(
    checkpoint_path=model_path,
    base_model=config['model']['base_model']
)

print("\n✓ Model loaded successfully")

Found checkpoints: ['checkpoint-464', 'checkpoint-522', 'checkpoint-580']
Loading latest checkpoint: models/checkpoints\checkpoint-580
Loading model from models/checkpoints\checkpoint-580


Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Loading weights:   0%|          | 0/288 [00:00<?, ?it/s]

✓ Loaded full model

✓ Model loaded successfully


In [13]:
# Cell 5: Load test data
test_path = config['data']['test_json']

with open(test_path, 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"Loaded {len(test_data)} test examples")
print("\nSample test example:")
print(json.dumps(test_data[0], indent=2, ensure_ascii=False))

Loaded 50 test examples

Sample test example:
{
  "idiom_en": "A.B.C",
  "idiom_si": "අයනු ආයනු",
  "meaning": "Something is very easy, simple, or requires little effort",
  "source_en": "He doesn't know even the A. B. C. of literary\ncriticism.",
  "target_si": "සාහිත්‍ය විවේචනයේ අයන්න වත් ඔහු නොදනියි.",
  "evaluation": "Yes"
}


In [14]:
# Cell 6: Extract source texts and generate translations
source_texts = [example['source_en'] for example in test_data]

# Generate translations in batches
print("Generating translations...")
predictions = batch_translate(
    texts=source_texts,
    model=model,
    tokenizer=tokenizer,
    src_lang=config['model']['source_lang'],
    tgt_lang=config['model']['target_lang'],
    max_length=config['training']['max_length'],
    batch_size=8
)

print(f"\n✓ Generated {len(predictions)} translations")

Generating translations...

✓ Generated 50 translations


In [20]:
print("="*80)
print("FULL DIAGNOSTIC")
print("="*80)

# 1. Check predictions
print("\n1. PREDICTIONS CHECK:")
print(f"   Total predictions: {len(predictions)}")
print(f"   Empty predictions: {sum(1 for p in predictions if not p)}")
print(f"   Average length: {sum(len(p) for p in predictions) / len(predictions):.1f} chars")

# 2. Check Sinhala content
import re
def has_sinhala(text):
    return bool(re.search(r'[\u0D80-\u0DFF]', text))

sinhala_count = sum(1 for p in predictions if has_sinhala(p))
print(f"\n2. LANGUAGE CHECK:")
print(f"   Predictions with Sinhala: {sinhala_count}/{len(predictions)}")

# 3. Check idiom matching
print(f"\n3. IDIOM MATCHING:")
matches = 0
for i, (ex, pred) in enumerate(zip(test_data, predictions)):
    if ex['idiom_si'] in pred:
        matches += 1
        
print(f"   Exact idiom matches: {matches}/{len(test_data)}")

# 4. Show 3 examples
print(f"\n4. SAMPLE OUTPUTS:")
for i in range(min(3, len(test_data))):
    print(f"\n   Example {i+1}:")
    print(f"   Expected idiom: {test_data[i]['idiom_si']}")
    print(f"   Prediction: {predictions[i][:100]}...")
    print(f"   Match: {test_data[i]['idiom_si'] in predictions[i]}")

print("\n" + "="*80)

FULL DIAGNOSTIC

1. PREDICTIONS CHECK:
   Total predictions: 50
   Empty predictions: 0
   Average length: 46.4 chars

2. LANGUAGE CHECK:
   Predictions with Sinhala: 50/50

3. IDIOM MATCHING:
   Exact idiom matches: 0/50

4. SAMPLE OUTPUTS:

   Example 1:
   Expected idiom: අයනු ආයනු
   Prediction: ඔහු සාහිත් ය විවේචනයේ උපාධිය පවා දන්නේ නැත....
   Match: False

   Example 2:
   Expected idiom: අත් හිටලා
   Prediction: ඒ කාරණය දැන් වසර ගණනාවක් තිස්සේ සැකසී තිබේ....
   Match: False

   Example 3:
   Expected idiom: වෙන කිසිත් නැතත්
   Prediction: සියල්ලටත් වඩා සත් යවාදී වන්න....
   Match: False



In [15]:
# Cell 7: Display comparisons for first 10 examples
print("=" * 80)
print("TRANSLATION COMPARISONS")
print("=" * 80)

for i in range(min(10, len(test_data))):
    example = test_data[i]
    prediction = predictions[i]
    
    print(f"\n--- Example {i+1} ---")
    print(f"English Idiom: {example['idiom_en']}")
    print(f"Sinhala Idiom: {example['idiom_si']}")
    print(f"\nSource (EN): {example['source_en']}")
    print(f"\nExpected (SI): {example['target_si']}")
    print(f"\nPredicted (SI): {prediction}")
    
    # Quick check if Sinhala idiom is present
    idiom_present = example['idiom_si'] in prediction
    print(f"\nIdiom Correct: {'✅ Yes' if idiom_present else '❌ No'}")
    print("-" * 80)

TRANSLATION COMPARISONS

--- Example 1 ---
English Idiom: A.B.C
Sinhala Idiom: අයනු ආයනු

Source (EN): He doesn't know even the A. B. C. of literary
criticism.

Expected (SI): සාහිත්‍ය විවේචනයේ අයන්න වත් ඔහු නොදනියි.

Predicted (SI): ඔහු සාහිත් ය විවේචනයේ උපාධිය පවා දන්නේ නැත.

Idiom Correct: ❌ No
--------------------------------------------------------------------------------

--- Example 2 ---
English Idiom: In abeyance
Sinhala Idiom: අත් හිටලා

Source (EN): That matter has now been <IDIOM>in abeyance</IDIOM> for a
number of years.

Expected (SI): ඒ කරුණ දැන් අවුරුදු ගණනකට අත් හිටලාය.

Predicted (SI): ඒ කාරණය දැන් වසර ගණනාවක් තිස්සේ සැකසී තිබේ.

Idiom Correct: ❌ No
--------------------------------------------------------------------------------

--- Example 3 ---
English Idiom: Above all
Sinhala Idiom: වෙන කිසිත් නැතත්

Source (EN): <IDIOM>Above all</IDIOM>, be truthful.

Expected (SI): වෙන කිසිත් නැතත්, සත්‍යවාදී වනු.

Predicted (SI): සියල්ලටත් වඩා සත් යවාදී වන්න.

Idiom Correct: ❌ 

In [17]:
# Cell 8: Create a DataFrame for easy viewing
results_df = pd.DataFrame({
    'Idiom_EN': [ex['idiom_en'] for ex in test_data],
    'Idiom_SI': [ex['idiom_si'] for ex in test_data],
    'Source': [ex['source_en'] for ex in test_data],
    'Reference': [ex['target_si'] for ex in test_data],
    'Prediction': predictions,
    'Idiom_Found': [ex['idiom_si'] in pred for ex, pred in zip(test_data, predictions)]
})

# Display summary
print("\n=== Quick Summary ===")
print(f"Total examples: {len(results_df)}")
print(f"Idiom found in translation: {results_df['Idiom_Found'].sum()} / {len(results_df)}")
print(f"Idiom accuracy: {results_df['Idiom_Found'].sum() / len(results_df) * 100:.1f}%")

# Show first few rows
print("\nFirst 5 results:")
results_df.head()


=== Quick Summary ===
Total examples: 50
Idiom found in translation: 0 / 50
Idiom accuracy: 0.0%

First 5 results:


,Idiom_EN,Idiom_SI,Source,Reference,Prediction,Idiom_Found
0,A.B.C,අයනු ආයනු,He doesn't know even the A. B. C. of literary\...,සාහිත්‍ය විවේචනයේ අයන්න වත් ඔහු නොදනියි.,ඔහු සාහිත් ය විවේචනයේ උපාධිය පවා දන්නේ නැත.,False
1,In abeyance,අත් හිටලා,That matter has now been <IDIOM>in abeyance</I...,ඒ කරුණ දැන් අවුරුදු ගණනකට අත් හිටලාය.,ඒ කාරණය දැන් වසර ගණනාවක් තිස්සේ සැකසී තිබේ.,False
2,Above all,වෙන කිසිත් නැතත්,"<IDIOM>Above all</IDIOM>, be truthful.","වෙන කිසිත් නැතත්, සත්‍යවාදී වනු.",සියල්ලටත් වඩා සත් යවාදී වන්න.,False
3,Above-board,සඟවන්න්ට දෙයක් නැති,His conduct has been entirely <IDIOM>above-boa...,ඔහුගේ හැසිරීම පිළිබඳව සැක කිරීමට හේතුවක් නැත.,ඔහුගේ හැසිරීම මුළුමනින්ම මධ් යස්ථව තිබී තිබේ.,False
4,Terms of abuse,ආක්‍රෝශ පරිසව - වචන,He constantly addressed her in <IDIOM>terms of...,ඔහු නිරන්තරයෙන් ඇයට ආක්‍රෝශ පරිභව වචනයෙන්\nකථා...,ඔහු නිතරම ඇයව අපයෝජනය කර ගනිමින් කතා කෙළේය.,False


In [18]:
# Cell 9: Prepare results for saving
results = []
for i, (example, prediction) in enumerate(zip(test_data, predictions)):
    result = {
        'example_id': i,
        'idiom_en': example['idiom_en'],
        'idiom_si': example['idiom_si'],
        'source_en': example['source_en'],
        'reference_si': example['target_si'],
        'prediction_si': prediction,
        'idiom_present': example['idiom_si'] in prediction
    }
    results.append(result)

# Save to JSON
output_path = Path(config['outputs']['predictions'])
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✓ Results saved to {output_path}")
print(f"  Total predictions: {len(results)}")


✓ Results saved to outputs\predictions\test_results.json
  Total predictions: 50


In [19]:
# Cell 10: Show some successful and unsuccessful translations
successful = results_df[results_df['Idiom_Found'] == True].head(3)
unsuccessful = results_df[results_df['Idiom_Found'] == False].head(3)

print("=== Successful Translations (Idiom Found) ===")
for idx, row in successful.iterrows():
    print(f"\nIdiom: {row['Idiom_EN']} → {row['Idiom_SI']}")
    print(f"Prediction: {row['Prediction'][:100]}...")

print("\n\n=== Unsuccessful Translations (Idiom Not Found) ===")
for idx, row in unsuccessful.iterrows():
    print(f"\nIdiom: {row['Idiom_EN']} → {row['Idiom_SI']}")
    print(f"Expected: {row['Reference'][:100]}...")
    print(f"Got: {row['Prediction'][:100]}...")

=== Successful Translations (Idiom Found) ===


=== Unsuccessful Translations (Idiom Not Found) ===

Idiom: A.B.C → අයනු ආයනු
Expected: සාහිත්‍ය විවේචනයේ අයන්න වත් ඔහු නොදනියි....
Got: ඔහු සාහිත් ය විවේචනයේ උපාධිය පවා දන්නේ නැත....

Idiom: In abeyance → අත් හිටලා
Expected: ඒ කරුණ දැන් අවුරුදු ගණනකට අත් හිටලාය....
Got: ඒ කාරණය දැන් වසර ගණනාවක් තිස්සේ සැකසී තිබේ....

Idiom: Above all → වෙන කිසිත් නැතත්
Expected: වෙන කිසිත් නැතත්, සත්‍යවාදී වනු....
Got: සියල්ලටත් වඩා සත් යවාදී වන්න....
